In [1]:
import pandas as pd
train=pd.read_csv("./input/train.csv")
test=pd.read_csv("./input/test.csv")
total=pd.concat([train,test],ignore_index=True)
len(train),len(test),len(total)

(159571, 153164, 312735)

In [2]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [3]:
test.tail()

,id,comment_text
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."
153163,ffffce3fb183ee80,""" \n :::Stop already. Your bullshit is not wel..."


In [4]:
total.tail()

,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic
312730,". \n i totally agree, this stuff is nothing bu...",fffcd0960ee309b5,NaN,NaN,NaN,NaN,NaN,NaN
312731,== Throw from out field to home plate. == \n\n...,fffd7a9a6eb32c16,NaN,NaN,NaN,NaN,NaN,NaN
312732,""" \n\n == Okinotorishima categories == \n\n I ...",fffda9e8d6fafa9e,NaN,NaN,NaN,NaN,NaN,NaN
312733,""" \n\n == """"One of the founding nations of the...",fffe8f1340a79fc2,NaN,NaN,NaN,NaN,NaN,NaN
312734,""" \n :::Stop already. Your bullshit is not wel...",ffffce3fb183ee80,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train.dtypes

id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

In [6]:
#from sklearn.metrics import make_scorer
#from sklearn.metrics import roc_auc_score
#auc_scorer=make_scorer(roc_auc_score, average='macro')

In [7]:
import nltk
stemmer=nltk.stem.PorterStemmer()
#stemmer.stem(vect.get_feature_names())

In [8]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#vect=CountVectorizer(min_df=5,max_df=0.1)

vect = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=30000)


In [9]:
vect.fit(train.comment_text)
X_train=vect.transform(train.comment_text)          
X_test=vect.transform(test.comment_text)  
y_col=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y_train=train[y_col]
X_train.shape,y_train.shape

/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


((159571, 30000), (159571, 6))

In [11]:
param_grid={'C':[1,]}
        

grid_y=[]
for y in y_col:
    print(y)
#    regr=LogisticRegression()
    y_train=train[y]
    regr=LogisticRegression(solver='sag')

    grid=GridSearchCV(regr,param_grid,cv=3,scoring= 'roc_auc',verbose=1)
    
    grid.fit(X_train,y_train)
    grid_y.append(grid)
    print(grid.best_score_, grid.best_params_)
    #regr_y.append(regr)
    #score=(cross_val_score(regr,X_train,y_train,cv=2,scoring= 'roc_auc')).mean()
    #print(score)
    #scores.append(score)

toxic
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.6s finished


0.9695194589661286 {'C': 1}
severe_toxic
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.8s finished


0.985333063057211 {'C': 1}
obscene
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

0.9820988513970366 {'C': 1}
insult
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.9s finished


0.976118233811078 {'C': 1}
identity_hate
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.4s finished


0.9745221406710637 {'C': 1}


In [13]:
import numpy as np
np.mean( [grid.best_score_ for grid in grid_y])

0.9787947814502035

In [14]:
for y,grid in zip(y_col,grid_y):
    y_train=train[y]
    print(y)
    grid.best_estimator_.fit(X_train,y_train)


toxic
severe_toxic
obscene
threat
insult
identity_hate


In [15]:
for y,grid in zip(y_col,grid_y):
    print(y)
    y_pred= grid.best_estimator_.predict_proba(X_test)
    test[y]=y_pred[:,1]

toxic
severe_toxic
obscene
threat
insult
identity_hate


In [16]:
test.to_csv('log2.csv',index=False,columns=['id']+y_col)

In [17]:
test

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,0.998024,0.172922,0.995809,0.029172,0.926205,0.237515
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,0.010010,0.003140,0.005382,0.001826,0.008129,0.003390
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0.010828,0.001640,0.005133,0.000880,0.007639,0.002248
3,00017563c3f7919a,":If you have a look back at the source, the in...",0.006133,0.002500,0.004479,0.001231,0.005295,0.001330
4,00017695ad8997eb,I don't anonymously edit articles at all.,0.048126,0.003354,0.013141,0.002069,0.021077,0.003303
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0.008802,0.002022,0.005097,0.001070,0.008009,0.001903
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...,0.008350,0.001655,0.005434,0.000860,0.007326,0.001959
7,000247e83dcc1211,:Dear god this site is horrible.,0.577273,0.003843,0.043014,0.003130,0.109558,0.005751
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ...",0.023590,0.002713,0.014561,0.002115,0.014907,0.003838
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...,0.009843,0.001420,0.005469,0.001129,0.008388,0.001781
